# Regression - LightGBM- Coral Triangle

## Notebook Setup

### Import Libraries

In [ ]:
# Import Standard Libraries
import os
import datetime
import pickle
import pandas as pd
import numpy as np

# Import Visualization Libraries
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import shap

In [ ]:
# SKLearn Pipeline Imports
from sklearn import set_config
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.base import BaseEstimator, TransformerMixin
#from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures
#from sklearn.preprocessing import FunctionTransformer, PowerTransformer, RobustScaler, QuantileTransformer

In [ ]:
# Multicolinearity Imports
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [ ]:
# Pandas Configs
pd.set_option('mode.chained_assignment', None)
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Ignore Warnings
import warnings
warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=UserWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# MapBox Token for Plotly Maps
px.set_mapbox_access_token(os.environ.get("MAPBOX_TOKEN"))

# Scikit Learn Configs
set_config(transform_output="pandas")
set_config(display="diagram")

### Import Data

In [ ]:
# GLOB Data
X_train_GLOB = pd.read_parquet("/work/data/Xy_Data/X_train_GLOB.parquet")
X_val_GLOB = pd.read_parquet("/work/data/Xy_Data/X_val_GLOB.parquet")
X_holdout_GLOB = pd.read_parquet("/work/data/Xy_Data/X_holdout_GLOB.parquet")

y_train_GLOB = pd.read_parquet("/work/data/Xy_Data/y_train_GLOB.parquet")
y_val_GLOB = pd.read_parquet("/work/data/Xy_Data/y_val_GLOB.parquet")
y_holdout_GLOB = pd.read_parquet("/work/data/Xy_Data/y_holdout_GLOB.parquet")

# CARB Data
X_train_CARB = pd.read_parquet("/work/data/Xy_Data/X_train_CARB.parquet")
X_val_CARB = pd.read_parquet("/work/data/Xy_Data/X_val_CARB.parquet")
X_holdout_CARB = pd.read_parquet("/work/data/Xy_Data/X_holdout_CARB.parquet")

y_train_CARB = pd.read_parquet("/work/data/Xy_Data/y_train_CARB.parquet")
y_val_CARB = pd.read_parquet("/work/data/Xy_Data/y_val_CARB.parquet")
y_holdout_CARB = pd.read_parquet("/work/data/Xy_Data/y_holdout_CARB.parquet")

# SEAA Data
X_train_SEAA = pd.read_parquet("/work/data/Xy_Data/X_train_SEAA.parquet")
X_val_SEAA = pd.read_parquet("/work/data/Xy_Data/X_val_SEAA.parquet")
X_holdout_SEAA = pd.read_parquet("/work/data/Xy_Data/X_holdout_SEAA.parquet")

y_train_SEAA = pd.read_parquet("/work/data/Xy_Data/y_train_SEAA.parquet")
y_val_SEAA = pd.read_parquet("/work/data/Xy_Data/y_val_SEAA.parquet")
y_holdout_SEAA = pd.read_parquet("/work/data/Xy_Data/y_holdout_SEAA.parquet")

GLOB_X = [X_train_GLOB, X_val_GLOB, X_holdout_GLOB]
GLOB_Y = [y_train_GLOB, y_val_GLOB, y_holdout_GLOB]

SEAA_X = [X_train_SEAA, X_val_SEAA, X_holdout_SEAA]
SEAA_Y = [y_train_SEAA, y_val_SEAA, y_holdout_SEAA]

CARB_X = [X_train_CARB, X_val_CARB, X_holdout_CARB]
CARB_Y = [y_train_CARB, y_val_CARB, y_holdout_CARB]

## Create the model pipeline

### Define the custom data transformer

In [ ]:
class DataTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):

        ##########
        # Define the deep copy so it leaves the original in place
        Xc = X.copy(deep=True)


        ##########
        # Define complex features based on Feature Importance and Interactions
        epsilon = 0.0000001 # Avoid division by zero
        Xc['Fishing_Inter'] = Xc['EN.FSH.THRD.NO'] * Xc['ER.FSH.CAPT.MT'] # Multiplicative effect of fishing and threatened species
        Xc['Fertilizer_Inter'] = Xc['AG.LND.AGRI.K2'] / (Xc['AG.CON.FERT.ZS'] + epsilon) # Proportional effect of agricultural land and fertilizer
        Xc['Fert_and_Turbidity_Interaction'] = Xc['Fertilizer_Inter'] * Xc['Turbidity']
        Xc['Dist_to_Shore_and_Turbidity_Interaction'] = Xc['Distance_to_Shore'] * Xc['Turbidity']

        # ##########
        # Define Thermal Interaction Features 
        # Polynomial:
        Xc['TSA_squared'] = Xc['TSA'] ** 2
        Xc['SSTA_squared'] = Xc['SSTA'] ** 2
        Xc['SSTA_cubed'] = Xc['SSTA'] ** 3
        Xc['TSA_cubed'] = Xc['TSA'] ** 3        
        Xc['TSA_cubed_SSTA'] = Xc['TSA_cubed'] * Xc['SSTA']
        Xc['TSA_cubed_SSTA_squared'] = Xc['TSA_cubed'] * Xc['SSTA_squared']


        # ########## 
        # Define columns to be kept
        keep_cols = [
            'Depth_m',
            'Distance_to_Shore',
            'Longitude_Degrees',
            'Turbidity',
            'Latitude_Degrees',
            'ClimSST',
            'Cyclone_Frequency',
            'SSTA_Frequency',
            'TSA_Frequency',
            'Fertilizer_Inter',
            'Fishing_Inter',
            'Dist_to_Shore_and_Turbidity_Interaction',
            'Fert_and_Turbidity_Interaction',
            'TSA_cubed_SSTA',
            'TSA_cubed_SSTA_squared',
            'SSTA','TSA','SSTA_DHW','TSA_DHW'
            ]             


        ##########
        # Select columns
        Xc = Xc[keep_cols]


        ##########
        # Prep for output
        # Sort the column names so the modeling process doesn't complain
        Xc = Xc.sort_index(axis=1)

        
        # Return the transformed dataframe
        return Xc

### Apply the data transformer

In [ ]:
pre_pipe = Pipeline([
            ('DataTransformer', DataTransformer()),
            # ('Scaler', StandardScaler()),
        ])

pre_pipe

In [ ]:
def process_data(loc, X_list, y_list, scaled=False, path='/work/data/Xy_Data/'):
    
    # Choose which pipeline object to use
    if scaled:
        pre_pipe = Pipeline([
            ('DataTransformer', DataTransformer()),
            ('Scaler', StandardScaler()),
        ])
    else:
        pre_pipe = Pipeline([
            ('DataTransformer', DataTransformer()),
            # ('Scaler', StandardScaler()),
        ])
    
    X_train_trans = pre_pipe.fit_transform(X_list[0], y_list[0])
    X_val_trans = pre_pipe.transform(X_list[1])
    X_holdout_trans = pre_pipe.transform(X_list[2])

    X_train_trans.to_parquet(f"{path}X_train_trans_{loc}.parquet")
    X_val_trans.to_parquet(f"{path}X_val_trans_{loc}.parquet")
    X_holdout_trans.to_parquet(f"{path}X_holdout_trans_{loc}.parquet")

    return f"{loc} processed and written to {path}"

# X_holdout_trans = pre_pipe.fit_transform(X_holdout, y_holdout)
# X_train_trans = pre_pipe.fit_transform(X_train, y_train)
# y_train_trans = y_train.loc[X_train_trans.index]
# y_holdout_trans = y_holdout.loc[X_holdout_trans.index]'

In [ ]:
process_data('GLOB', GLOB_X, GLOB_Y)
process_data('SEAA', SEAA_X, SEAA_Y)
process_data('CARB', CARB_X, CARB_Y)

### Create a Pearson Correlation Matrix to review collinearity of the data

In [ ]:
# rehydrate the x and y dataframes for the correlation matrix
df = pd.concat([X_train_trans, y_train_trans], axis=1)

# Create the correlation matrix
corr_matrix = df.corr(method='pearson')
corr_mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Plot the matrix
plt.figure(figsize=(25, 25))  # Adjust the figure size as needed
sns.heatmap(corr_matrix, mask=corr_mask, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, square=True, cbar_kws={"shrink": 0.8})

plt.yticks(rotation=0)
plt.xticks(rotation=45, ha='right')
plt.title('Pearson Correlation Matrix')

plt.show()

### Perform multicollinearity check using Variance Inflation Factor

In [ ]:
# VIF needs an intercept term in the model, so add a constant to X_train_trans
X_with_const = add_constant(X_train_trans)

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = X_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(len(X_with_const.columns))]

# Print VIF for each feature in descending order, skipping the constant
print(vif_data[1:].sort_values(by='VIF', ascending=False))